In [2]:
import numpy as np
import os
from PIL import Image

In [3]:
#AMF
def rgb2gray(rgb):
    if(len(rgb.shape) == 3):
        return np.uint8(np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]))
    else:
        return rgb

def calculate_median(array):
    """Return the median of 1-d array"""
    sorted_array = np.sort(array) #timsort (O(nlogn))
    median = sorted_array[len(array)//2]
    return median

def level_A(z_min, z_med, z_max, z_xy, S_xy, S_max):
    if(z_min < z_med < z_max):
        return level_B(z_min, z_med, z_max, z_xy, S_xy, S_max)
    else:
        S_xy += 2 #increase the size of S_xy to the next odd value.
        if(S_xy <= S_max): #repeat process
            return level_A(z_min, z_med, z_max, z_xy, S_xy, S_max)
        else:
            return z_med
        
def level_B(z_min, z_med, z_max, z_xy, S_xy, S_max):
    if(z_min < z_xy < z_max):
        return z_xy
    else:
        return z_med
    
def amf(image, initial_window, max_window):
    """runs the AMF process on an image"""
    xlength, ylength = image.shape #get the shape of the image.
    
    z_min, z_med, z_max, z_xy = 0, 0, 0, 0
    S_max = max_window
    S_xy = initial_window #dynamically to grow
    
    output_image = image.copy()
    
    for row in range(S_xy, xlength-S_xy-1):
        for col in range(S_xy, ylength-S_xy-1):
            filter_window = image[row - S_xy : row + S_xy + 1, col - S_xy : col + S_xy + 1] #filter window
            target = filter_window.reshape(-1) #make 1-dimensional
            z_min = np.min(target) #min of intensity values
            z_max = np.max(target) #max of intensity values
            z_med = calculate_median(target) #median of intensity values
            z_xy = image[row, col] #current intensity
            
            #Level A & B
            new_intensity = level_A(z_min, z_med, z_max, z_xy, S_xy, S_max)
            output_image[row, col] = new_intensity
    return output_image

In [6]:
PATH = 'MLRSNet Dataset\\MLRSNet for Semantic Scene Understanding\\Images' #TODO Change if Necessary

NUM_CLASSES = 22 #TODO Change if needed
NUM_DATAPOINTS = 7000 #TODO Change if needed
num_each_class = int(np.ceil(NUM_DATAPOINTS / NUM_CLASSES)) #304

for FOLDER in os.listdir(PATH):
    os.mkdir(f"MLRSNet Dataset\\MLRSNet for Semantic Scene Understanding\\AMFImages_new\\{FOLDER}")
    files = os.listdir(f"MLRSNet Dataset\\MLRSNet for Semantic Scene Understanding\\Images\\{FOLDER}")
    for file in files[304:num_each_class+50]:
        image_orig = Image.open(f"MLRSNet Dataset\\MLRSNet for Semantic Scene Understanding\\Images\\{FOLDER}\\{file}")
        grayscale_image = rgb2gray(np.array(image_orig)) #outputs a grayscaled image
        output = amf(grayscale_image, initial_window=3, max_window=7)
        amf_img = Image.fromarray(output)
        amf_img.save(f"MLRSNet Dataset\\MLRSNet for Semantic Scene Understanding\\AMFImages_new\\{FOLDER}\\{file}")